In [1]:
import torch
import torch.nn as nn
import pandas as pd
import os
import numpy as np
from torch.utils.data import DataLoader
import tokenizers

In [2]:
# read the datasets. They are divided in test and train
train = pd.read_csv('../data/train.csv')
print(f'Lenght of train data is {len(train):,} observations')
# drop missing values
train = train.dropna()
print(f'Lenght of train data is {len(train):,} observations')
train.head(7)

Lenght of train data is 27,481 observations
Lenght of train data is 27,480 observations


textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   
5  28b57f3990  http://www.dothebouncy.com/smf - some shameles...   
6  6e0c6d75b1  2am feedings for the baby are fun when he is a...   

                                       selected_text sentiment  
0                I`d have responded, if I were going   neutral  
1                                           Sooo SAD  negative  
2                                        bullying me  negative  
3                                     leave me alone  negative  
4                                      Sons of ****,  negative  
5  http://www.dothebouncy.com/smf - some shameles...   neutral  
6                                                fun  positive

In [3]:
#train = train.iloc[0:300]
# split dataset into training and validation 
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(train,
                              random_state = 42,
                              stratify = train[['sentiment']],
                              test_size = 0.1)

In [4]:
# repeat for test
test = pd.read_csv('../data/test.csv')
print(f'Lenght of the test data is {len(test):,} observations')

Lenght of the test data is 3,534 observations


In [5]:
from tokenizers import BertWordPieceTokenizer
tokenizer_manual = BertWordPieceTokenizer('../models_trained/bert-base-uncased-vocab.txt', lowercase=True)

In [6]:
# test
from transformers import BertTokenizerFast
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
max_seq_length = 180

class Tweet_Loader(object):
    '''
    1. tokenize the text
    2. find the indices of the selected text in the original text
    3. append the question to the original text and modify original indexes
    4. identify start index and end index
    5. identify start and end index
    '''

    def __init__(self, text, selected_text, sentiment, textID):
        
        # define the text column from the dataframe
        self.text_column = text.tolist()
    
        # define sentiment
        self.sentinent = sentiment.tolist()
        
        # define selected text
        self.selected_text = selected_text.tolist()
    
        # define the label column and transform it to list
        self.textID = textID.tolist()
        
    # iter method to get each element at the time and tokenize it using bert        
    def __getitem__(self, index):
        
        sentinent = self.sentinent[index]
        text_id = self.textID[index]
        text = self.text_column[index]
        selected_text = self.selected_text[index]
        
        # clean text and selected text
        text = " ".join(str(text).split())
        selected_text = " ".join(str(selected_text).split())
        
        # find beggining of character
        #target_char_idx_start = text.index(selected_text)
        #target_char_idx_end = target_char_idx_start + len(selected_text)
        #print(text, selected_text, '', target_char_idx_start, target_char_idx_end)
        # get tokenized text
        tokenized_text = tokenizer.tokenize(text)
        tokenized_selected_text = tokenizer.tokenize(selected_text)
        #tokenized_text = tokenizer_manual.encode(text).tokens
        #tokenized_selected_text = tokenizer_manual.encode(selected_text).tokens
        
        # find the spans of the answer
        start_span_index = -1
        end_span_index = -1
        
        len_sel_text = len(tokenized_selected_text)
        
        # define the start and end index of the selected text. We iterate over the original tweet and
        # identify the point where the first word of the original text matches the first word of the selected
        # text. Then if traversing the substring of original tweet is the same as the selected tweet
        # then get the start and end spans
        for index in (i for i, e in enumerate(tokenized_text) if e == tokenized_selected_text[0]):
            if tokenized_text[index: index+len_sel_text] == tokenized_selected_text:
                start_span_index = index
                end_span_index = index + len_sel_text - 1
                break
                
        # use the encode plus function
        dictionary_inputs = tokenizer.encode_plus(sentinent, text, 
                                                  max_length = max_seq_length,
                                                  #truncation_strategy  ='do_not_truncate', 
                                                  add_special_tokens = True,
                                                  pad_to_max_length = True, padding_side='Right', 
                                                  is_pretokenized = False, return_token_type_ids=True,
                                                  return_attention_mask=True,
                                                  return_offsets_mapping = True,
                                                  return_tensors = 'pt')
        
        # create the input by adding the CLS token, the SEP token for separaring the question and the end of the
        # whole sentence
        #input_sentence = ['CLS'] + [sentinent] + ['SEP'] + tokenized_text + ['SEP']
        #inp=dictionary_inputs.get('input_ids')
        #inp = inp[0].tolist()
        #offsets = dictionary_inputs.get('offset_mapping')[0]
        start_span_index += 3
        end_span_index +=  3
        
        #tweet_offsets = dictionary_inputs.get('offset_mapping')[0]
        #filtered_output = ''
        #for ix in range(start_span_index, end_span_index):
            #print(text[tweet_offsets[ix][0]:tweet_offsets[ix][1]])
        #    filtered_output += text[tweet_offsets[ix][0]:tweet_offsets[ix][1]]
        #    if (ix+1) < len(tweet_offsets) and tweet_offsets[ix][1] < tweet_offsets[ix+1][0]:
        #        filtered_output += " "
                    
        #print(f'original was:{text}, and selected: {selected_text}',
        #      f'what we get back: {tokenizer.decode(inp[start_span_index:end_span_index+1])}',
        #      f'and f output {filtered_output}')
        
        return [dictionary_inputs, start_span_index, end_span_index, text_id, selected_text ]
  
    def __len__(self):
        return len(self.text_column)

In [7]:
#train_data
#example[0].keys()

In [8]:
#print(train['text'].iloc[12], train['selected_text'].iloc[12])
#targets_(train['text'].iloc[12], train['selected_text'].iloc[12])
#print(tokenizer.decode([20754]))

In [9]:
# create a class to process the traininga and test data
batch_size = 4
training_data = Tweet_Loader(train_data['text'],
                             train_data['selected_text'], 
                             train_data['sentiment'],
                             train_data['textID'])

valid_data =  Tweet_Loader(val_data['text'],val_data['selected_text'], val_data['sentiment'], val_data['textID'])

# use the dataloaders class to load the data
dataloaders_dict = {'train': DataLoader(training_data, batch_size=batch_size, shuffle=False, num_workers=4),
                    'val': DataLoader(valid_data, batch_size=batch_size, shuffle=True, num_workers=4)
                   }

dataset_sizes = {'train':len(training_data),
                 'val':len(valid_data)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [10]:
'''
from torch.utils.data import Dataset, DataLoader

a = Tweet_Loader(train['text'],train['selected_text'], train['sentiment'], train['textID'])

# use the dataloaders class to load the data
dataloaders_dict = {'train': DataLoader(a, batch_size=1, shuffle=True, num_workers=4)}

dataset_sizes = {'train':len(a)}
'''

"\nfrom torch.utils.data import Dataset, DataLoader\n\na = Tweet_Loader(train['text'],train['selected_text'], train['sentiment'], train['textID'])\n\n# use the dataloaders class to load the data\ndataloaders_dict = {'train': DataLoader(a, batch_size=1, shuffle=True, num_workers=4)}\n\ndataset_sizes = {'train':len(a)}\n"

In [11]:
#train.loc[44]['text']
example = next(iter(dataloaders_dict.get('train')))
#tokenizer_manual.decode(example[0].get('input_ids')[2][0].tolist())
#example[0].get('input_ids')[0][0].tolist()
#[example[1][1].item():example[2][1].item()+1]

#print(example[1],example[2])

In [12]:
#print(example[0].get('offset_mapping')[0][0])

In [13]:
"""#example[4]
#example[0].get('input_ids')[0][0][example[1][0].item():example[2][0].item()+1]
#tokenizer_manual.decode(skip_special_tokens=True)
#import tokenizers.decoders
def is_subtoken(word):
    if word[:2] == "##":
        return True
    else:
        return False

def original_string(tokens):
    tokens = ['why', 'isn', "##'", '##t', 'Alex', "##'", 'text', 'token', '##izing']
    restored_text = []
    for i in range(len(tokens)):
        if not is_subtoken(tokens[i]) and (i+1)<len(tokens) and is_subtoken(tokens[i+1]):
            restored_text.append(tokens[i] + tokens[i+1][2:])
            if (i+2)<len(tokens) and is_subtoken(tokens[i+2]):
                restored_text[-1] = restored_text[-1] + tokens[i+2][2:]
        elif not is_subtoken(tokens[i]):
            restored_text.append(tokens[i])
    return restored_text
"""

'#example[4]\n#example[0].get(\'input_ids\')[0][0][example[1][0].item():example[2][0].item()+1]\n#tokenizer_manual.decode(skip_special_tokens=True)\n#import tokenizers.decoders\ndef is_subtoken(word):\n    if word[:2] == "##":\n        return True\n    else:\n        return False\n\ndef original_string(tokens):\n    tokens = [\'why\', \'isn\', "##\'", \'##t\', \'Alex\', "##\'", \'text\', \'token\', \'##izing\']\n    restored_text = []\n    for i in range(len(tokens)):\n        if not is_subtoken(tokens[i]) and (i+1)<len(tokens) and is_subtoken(tokens[i+1]):\n            restored_text.append(tokens[i] + tokens[i+1][2:])\n            if (i+2)<len(tokens) and is_subtoken(tokens[i+2]):\n                restored_text[-1] = restored_text[-1] + tokens[i+2][2:]\n        elif not is_subtoken(tokens[i]):\n            restored_text.append(tokens[i])\n    return restored_text\n'

In [14]:
#s = reconstruct_string(example[1][1].item(), example[2][1].item(), example[0].get('input_ids')[1][0])
#s = tokenizert_manual.example[0].get('input_ids')[1][0][example[1][1].item():example[2][1].item()+1].tolist(),
#                           skip_special_tokens=True)
#for i in example[0].get('input_ids')[0][0]:
#        print(tokenizer_manual.id_to_token(i))
#print(f'original text is  """{example[4][1]}""" and reconstructed is '
#      f'""" {s} """')


#example[0].keys()

In [15]:
def reconstruct_string(start_index, end_index, input_ids):
    # get the actual span tokes using the start_span and end_span
    #original_tokens = input_ids[start_index:end_index+1]
    original_tokens = tokenizer.decode(input_ids.tolist()[start_index:end_index+1])
    # move to string 
    #original_string = ' '.join([x for x in original_tokens])

    return original_tokens
    
a = reconstruct_string(example[1][0].item(), example[2][0].item(), 
                   example[0].get('input_ids')[0][0])
    

In [16]:
a

'morning, just checking you can still find your twhirl'

In [17]:
#tokens_ =  tokenizer.convert_ids_to_tokens(example[0].get('input_ids')[0][0][example[0][0]:example[2][0]])
#tokens_
#example[0].get('input_ids')[0][0]
#a= example[1][0].item()
#b= example[2][0].item()+1
#print(example[0].get('input_ids')[0][0][example[1][0].item():example[2][0].item()+1]
#     )
#text_r = ' '.join([x for x in tokens_])
#print(text_r)
#fine_text = text.replace(' ##', '')

In [18]:
#a = tokenizer.tokenize('sigh.. stayed up late cause was on a role w/ assessment, now I`m so tired my eyes r stinging & my head hurts.. & I still need 500 words')
#len(a)
# import the model
from transformers import BertForQuestionAnswering, BertConfig
config  = BertConfig()
model = BertForQuestionAnswering.from_pretrained(
    "bert-base-uncased", config = config)

In [19]:
# check the model
model

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [20]:
# define jaccard metric
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): 
        return 0.5
    c = a.intersection(b)
    score = float(len(c)) / (len(a) + len(b) - len(c))
    return score

In [21]:
#jaccard(s, example[4][1])

#print(s, example[6][1])
#tokenizer1.decode()
#example[0].get('input_ids')[0][0][example[1][0].item():example[2][0].item()+1]



In [22]:
'''
# ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')
    
'''

'\n# ======== Reconstruct Answer ========\n    # Find the tokens with the highest `start` and `end` scores.\n    answer_start = torch.argmax(start_scores)\n    answer_end = torch.argmax(end_scores)\n\n    # Get the string versions of the input tokens.\n    tokens = tokenizer.convert_ids_to_tokens(input_ids)\n\n    # Start with the first token.\n    answer = tokens[answer_start]\n\n    # Select the remaining answer tokens and join them with whitespace.\n    for i in range(answer_start + 1, answer_end + 1):\n        \n        # If it\'s a subword token, then recombine it with the previous token.\n        if tokens[i][0:2] == \'##\':\n            answer += tokens[i][2:]\n        \n        # Otherwise, add a space then the token.\n        else:\n            answer += \' \' + tokens[i]\n\n    print(\'Answer: "\' + answer + \'"\')\n    \n'

In [23]:
import logging
logging.basicConfig(level = logging.INFO, filename ='../logs/bert_tweet_extraction_160_nofinetune.txt', filemode ='w', 
                   format = '%(name)s -%(levelname)s - %(message)s')

In [24]:
import torch.optim as optim 
from torch.optim import lr_scheduler, AdamW
#criterion = nn.CrossEntropyLoss()


#lrmain = 1e-3

optim1 = optim.AdamW(model.parameters(), lr=3e-5, betas=(0.9, 0.999))
    #[
    #    {"params":model.bert.parameters(),"lr": lrmain},
   #])

optimizer_ft = optim1

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.1)

In [25]:
import torch.nn.functional as F
#import torch.nn.functional as F
import copy 
import time
number_steps = 10
print(number_steps)

def train_model(model, optimizer, scheduler, num_epochs=3):
    best_eval_jac = 0.0
    for epoch in range(num_epochs):
        #zero the model gradients
        model.zero_grad()
        print(f'starting epoch {epoch+1} out of {num_epochs}')
        training_loss = []
        training_jaccard = []
        val_loss = []
        val_jaccard = []
        outputs_ = []
        labels_ = []

        epoch_loss = 0.0

        # track number of batches and number of iterations
        counter = 0
        iterations = 0

        # Iterate over data, feeding inputs, attention masks and labels
        model.train()
        for i, (dict_param, start_position, end_position, tweet_id, selected_text) in enumerate(dataloaders_dict['train']):
            # add a counter that will register how many examples we have fed to the
            # model
            counter+= batch_size
            iterations+=1
            # move the sequences, labels and masks to the GPU
            inputs = dict_param.get('input_ids').to(device).squeeze()
            #print(inputs.shape)
            token_type_ids = dict_param.get('token_type_ids').to(device).squeeze()
            #print(token_type_ids.shape)
            attention_mask = dict_param.get('attention_mask').to(device).squeeze()
            #print(attention_mask.shape)
            start_pos = start_position.to(device)
            end_pos = end_position.to(device)
            

            # feed the sequences to the model, specifying the attention mask
            loss, start_scores, end_scores = model(inputs, token_type_ids = token_type_ids, 
                                                   attention_mask = attention_mask, 
                                                   start_positions = start_pos,
                                                   end_positions = end_pos)
            
            # 
            training_loss.append(loss.item())
            epoch_loss+=loss.item()
            loss.backward()
            #print(start_scores, start_scores.shape)
            
                        
            # update every number of accumulation
            if (i+1) % number_steps == 0:
                
                optimizer.step()                            # Now we can do an optimizer step
                model.zero_grad()                           # Reset gradients tensors
      
            # only present the information 
            if counter%100 == 0:
                
                start_preds=torch.argmax(start_scores,dim=1).cpu().detach().numpy()
                end_preds=torch.argmax(end_scores,dim=1).cpu().detach().numpy()
                start_index = start_pos.cpu().cpu().detach().numpy()
                end_index = end_pos.cpu().cpu().detach().numpy()
                inputs = inputs.cpu().detach().numpy()
                       
                for i in range(inputs.shape[0]):
                    # here we reconstruct the string with original index
                    original_string = reconstruct_string(start_index[i], end_index[i], inputs[i])
                    # repeat with what we found
                    pred_string = reconstruct_string(start_preds[i], end_preds[i], inputs[i])
                    #print(type(selected_text),type(pred_string))
                    jaccard_score = jaccard(selected_text[i],pred_string)
                    training_jaccard.append(jaccard_score)
                    #print(f'the original is {original_string}, the predicted is {pred_string}')
                    
                print(f'reconstruction is: {original_string} and original is: {selected_text[i]}')
                print(f'current loss is {epoch_loss/iterations} and jaccard',
                      f'is {np.mean(training_jaccard):,.2%}')
                
        with torch.no_grad():
            model.eval()
            counter_val = 0
            iterations_val = 0
            
            for i, (dict_param, start_position, end_position, tweet_id, selected_text) in enumerate(dataloaders_dict['train']):
            # add a counter that will register how many examples we have fed to the
            # model
                counter+= batch_size
                iterations_val+=1
            # move the sequences, labels and masks to the GPU
                inputs = dict_param.get('input_ids').to(device).squeeze()
                token_type_ids = dict_param.get('token_type_ids').to(device).squeeze()
                attention_mask = dict_param.get('attention_mask').to(device).squeeze()
                start_pos = start_position.to(device)
                end_pos = end_position.to(device)

            # feed the sequences to the model, specifying the attention mask
                loss, start_scores, end_scores = model(inputs, token_type_ids = token_type_ids, 
                                                   attention_mask = attention_mask, 
                                                   start_positions = start_pos,
                                                   end_positions = end_pos)
            
                val_loss.append(loss.item())
                
                if counter%10 == 0:

                    start_preds=torch.argmax(start_scores,dim=1).cpu().detach().numpy()
                    end_preds=torch.argmax(end_scores,dim=1).cpu().detach().numpy()
                    start_index = start_pos.cpu().cpu().detach().numpy()
                    end_index = end_pos.cpu().cpu().detach().numpy()
                    inputs = inputs.cpu().detach().numpy()
                       
                    for i in range(inputs.shape[0]):
                        #original_string = reconstruct_string(start_index[i], end_index[i], inputs[i])
                        pred_string = reconstruct_string(start_preds[i], end_preds[i], inputs[i])
                        print(selected_text[i],pred_string)
                        jaccard_score = jaccard(selected_text[i],pred_string)
                        val_jaccard.append(jaccard_score)
                        
                
                        print(f'current validation loss is {np.sum(val_loss)/iterations_val} and'
                              f'jaccard score is {np.mean(val_jaccard):,.2%}')
                              
        print(f'For epoch {epoch+1} training loss is {np.sum(training_loss)/iterations}, \
        training accuracy is {np.mean(training_jaccard):,.2%}, Validation \
        loss is {np.sum(val_loss)/iterations_val} and validation accuracy is {np.mean(val_jaccard):,.2%}')
        jac_acc = np.mean(val_jaccard)
        if jac_acc >= best_eval_jac:
            best_eval_jac = jac_acc
            print(f'saving the model with jac_acc  {jac_acc:,.2%} ')
            torch.save(model.state_dict(), '../models_trained/bert_tweet_extraction_no_finetuned.pth')
            #model.config.to_json_file('../models_trained/bert_base_uncased.json')
            torch.save(model, '../models_trained/bert_tweet_extraction_no_finetuned.bin')
            #torch.save(optimizer_ft.state_dict(), 
            #           'bert_tweet_classification_state_dict_280_no_finetuned_optimizer.pth')
            #model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
            #output_model_file = os.path.join("bert_tweet_classification_280_no_finetuned_model.bin")

        else:
            print(f'model did not improve')
        
        logging.info(f'We completed epoch {epoch+1} with a training loss of {np.sum(training_loss)/iterations} \
        a jac score of {np.mean(training_jaccard):,.2%}, Validation \
        loss is {np.sum(val_loss)/iterations_val} and validation accuracy is {np.mean(val_jaccard):,.2%}')
      
    return model

10


In [26]:
# time to train 
model.to(device)
model_ft1 = train_model(model, optimizer_ft, exp_lr_scheduler,
                       num_epochs=3)

starting epoch 1 out of 3
reconstruction is: it ` s after midnight & i hear the shower calling me - technically it ` s not twice in one day since it ` s morning - - i ` m not very'green '. and original is: It`s after midnight & I hear the shower calling me- Technically it`s not twice in one day since it`s morning-- I`m not very 'green'.
current loss is 5.147036838531494 and jaccard is 33.46%
reconstruction is: thanks and original is: thanks
current loss is 4.937373332977295 and jaccard is 21.42%
reconstruction is: disappointment really sucks! and original is: Disappointment really sucks!
current loss is 4.714729283650716 and jaccard is 15.11%
reconstruction is: hate and original is: hate
current loss is 4.442237493991851 and jaccard is 17.48%
reconstruction is: happy and original is: Happy
current loss is 4.191350318908691 and jaccard is 14.94%
reconstruction is: i need a code and original is: I need a code
current loss is 3.971504987080892 and jaccard is 14.88%
reconstruction is: love

reconstruction is: amazing and original is: amazing
current loss is 1.7917399113785988 and jaccard is 46.54%
reconstruction is: what ` s worse... and original is: what`s worse...
current loss is 1.7840041480614588 and jaccard is 46.12%
reconstruction is: i missed my favorite judge and now i can ` t sleep. and original is: I missed my favorite judge and now I can`t sleep.
current loss is 1.7689631500559033 and jaccard is 46.72%
reconstruction is: ohhh, how lovely! i ` m glad that you had a great time there. and original is: Ohhh, how lovely! I`m glad that you had a great time there.
current loss is 1.7595922742287318 and jaccard is 46.20%
reconstruction is: where is teng? and original is: Where is Teng?
current loss is 1.750946330352263 and jaccard is 46.72%
reconstruction is: sad and original is: sad
current loss is 1.7412804013277803 and jaccard is 47.29%
reconstruction is: guus ` s last game 2moz and original is: Guus`s last game 2moz
current loss is 1.73114416477973 and jaccard is 4

reconstruction is: knight online come back and original is: Knight Online Come Back
current loss is 1.455904829572923 and jaccard is 51.28%
reconstruction is: i want to know when the auditions are mander! text or... reply please! and original is: I want to know when the auditions are Mander! Text or...reply please!
current loss is 1.450966402444185 and jaccard is 51.27%
reconstruction is: we ` re awesome and original is: we`re awesome
current loss is 1.4457970665612268 and jaccard is 51.50%
reconstruction is: thanks and original is: thanks
current loss is 1.4414342177143464 and jaccard is 51.87%
reconstruction is: stupid, and original is: stupid,
current loss is 1.437088517325265 and jaccard is 52.10%
reconstruction is: thank and original is: thank
current loss is 1.4334097632372154 and jaccard is 52.11%
reconstruction is: recovering from my crazy family. i love them, but they ain ` t got it all i only see em a couple of times a year, so i guess i can deal. and original is: Recovering 

reconstruction is: i hate and original is: i hate
current loss is 1.3162474176883698 and jaccard is 52.38%
reconstruction is: one of my excellent designers and original is: One of my excellent designers
current loss is 1.3138834193842301 and jaccard is 52.45%
reconstruction is: having one of my bad days.. and original is: Having one of my bad days..
current loss is 1.3115104552632884 and jaccard is 52.50%
reconstruction is: it doesn ` t even have a scroll ball. and original is: it doesn`t even have a scroll ball.
current loss is 1.3118500546224756 and jaccard is 52.31%
reconstruction is: miss and original is: miss
current loss is 1.3098536763485376 and jaccard is 52.57%
reconstruction is: [SEP] and original is: One Thing I Hate About Friends An
current loss is 1.307035306138377 and jaccard is 52.76%
reconstruction is: i must have just missed you. i was down there until 9 : 40 or so. not sure if i ` ll make it down again and original is: i must have just missed you. I was down there unt

reconstruction is: fun and original is: fun
current loss is 1.2333391720718807 and jaccard is 54.06%
reconstruction is: microsoft has released zune also in europe, no hd for us and original is: Microsoft has released Zune also in Europe, no HD for us
current loss is 1.2323879742083237 and jaccard is 54.20%
reconstruction is: i accidentaly slammed my finger in the trunk and original is: I accidentaly slammed my finger in the trunk
current loss is 1.231235822147131 and jaccard is 54.11%
reconstruction is: ill in bed my stomach is killing me and original is: Ill in bed my stomach is killing me
current loss is 1.2297136575369099 and jaccard is 54.34%
reconstruction is: love and original is: love
current loss is 1.228398259777834 and jaccard is 54.50%
reconstruction is: dreary and original is: dreary
current loss is 1.227213793852059 and jaccard is 54.51%
reconstruction is: blogging - - http : / / 13tolife. us / mentioning links to contests where you might just win a free book or two! and o

...

All the people I talk to are GONE all the people i talk to are gone
current validation loss is 0.5275685018003599 andjaccard score is 71.14%
um... thats my favorite beer!! why must you torture me so when I can`t make it down there!! um... thats my favorite beer!! why must you torture me so when i can ` t make it down there!!
current validation loss is 0.5275685018003599 andjaccard score is 71.14%
**** * * * *
current validation loss is 0.5275685018003599 andjaccard score is 71.13%
r disappointing this year disappointing
current validation loss is 0.5275685018003599 andjaccard score is 71.12%
fun fun
current validation loss is 0.5275068012585672 andjaccard score is 71.12%
congratss to them congratss to them
current validation loss is 0.5275068012585672 andjaccard score is 71.13%
you are adorable you are adorable
current validation loss is 0.5275068012585672 andjaccard score is 71.14%
Nooo, I`m not in love. I`m just developing a crush. A small one. Heehee. My second one. Shadduppp. nooo,

rarely enough rarely enough
current validation loss is 0.5273322143272063 andjaccard score is 71.14%
Idk yet!!! _Gyrl I wanna go soon tho...but I want more tweeps to be there idk yet!!! _ gyrl i wanna go soon tho... but i want more tweeps to be there
current validation loss is 0.5273322143272063 andjaccard score is 71.14%
boo fo boo for all the season finales..
current validation loss is 0.5273322143272063 andjaccard score is 71.12%
blues for me please blues for me please
current validation loss is 0.5273322143272063 andjaccard score is 71.13%
nat has a twitter acct! nat has a twitter acct!
current validation loss is 0.527310778615204 andjaccard score is 71.14%
Well for one thing I might be branded a 'public menace' by the state. well for one thing i might be branded a'public menace'by the state.
current validation loss is 0.527310778615204 andjaccard score is 71.13%
It will if I do it in a round about way. I ahve to copy the public contacts into my personal contacts first. no big deal

Absolutely absolutely
current validation loss is 0.5263350050503485 andjaccard score is 71.24%
mum is driving me and the Eric out to jt for the night. mum is driving me and the eric out to jt for the night.
current validation loss is 0.5263350050503485 andjaccard score is 71.24%
Its funny funny
current validation loss is 0.5263350050503485 andjaccard score is 71.24%
e sore hate
current validation loss is 0.5263350050503485 andjaccard score is 71.22%
http://twitpic.com/4jcfg - About to cycle within this quarry on Shipley glen near Baildon moor before heading back through the woods http : / / twitpic. com / 4jcfg - about to cycle within this quarry on shipley glen near baildon moor before heading back through the woods
current validation loss is 0.5263214534335242 andjaccard score is 71.22%
loves surprises, but is never surprised loves surprises, but is never surprised
current validation loss is 0.5263214534335242 andjaccard score is 71.23%
i love him i love him
current validation loss i

just did my bit to stimulate the economy just did my bit to stimulate the economy
current validation loss is 0.5255606319406837 andjaccard score is 71.28%
sick. sick.
current validation loss is 0.5255606319406837 andjaccard score is 71.29%
bummed bummed
current validation loss is 0.5255606319406837 andjaccard score is 71.29%
loving loving
current validation loss is 0.5255606319406837 andjaccard score is 71.30%
good luck to them good luck to them
current validation loss is 0.5256044859210971 andjaccard score is 71.31%
poor poor
current validation loss is 0.5256044859210971 andjaccard score is 71.31%
_carter it says the video is private... and won`t let me watch _ carter it says the video is private... and won ` t let me watch
current validation loss is 0.5256044859210971 andjaccard score is 71.31%
****.. * * * *..
current validation loss is 0.5256044859210971 andjaccard score is 71.29%
i can`t see results for whyareyoustillhere i can ` t see results for whyareyoustillhere
current valida

loving th loving
current validation loss is 0.5252108647886504 andjaccard score is 71.27%
cool yay emily i m cool
current validation loss is 0.5252108647886504 andjaccard score is 71.26%
Time flies...and i never got it all right. Why oh why? time flies... and i never got it all right. why oh why?
current validation loss is 0.5252108647886504 andjaccard score is 71.26%
congradts on ur show Even tho i wasnt there lol congradts on ur show even tho i wasnt there lol
current validation loss is 0.5252108647886504 andjaccard score is 71.27%
Such a pretty baby! such a pretty baby!
current validation loss is 0.5251863043678452 andjaccard score is 71.28%
I can understand! i can understand!
current validation loss is 0.5251863043678452 andjaccard score is 71.28%
im almost to 300 updates. 200 more to goo im almost to 300 updates. 200 more to goo
current validation loss is 0.5251863043678452 andjaccard score is 71.29%
Missed missed
current validation loss is 0.5251863043678452 andjaccard score is 7

In [55]:
# test
max_seq_length = 180

class Tweet_Loader_Test(object):
    '''
    1. tokenize the text
    2. find the indices of the selected text in the original text
    3. append the question to the original text and modify original indexes
    4. identify start index and end index
    5. identify start and end index
    '''

    def __init__(self, text, sentiment, textID):
        
        # define the text column from the dataframe
        self.text_column = text.tolist()
    
        # define sentiment
        self.sentinent = sentiment.tolist()
                   
        # define the label column and transform it to list
        self.textID = textID.tolist()
        
    # iter method to get each element at the time and tokenize it using bert        
    def __getitem__(self, index):
        
        sentinent = self.sentinent[index]
        text_id = self.textID[index]
        text = self.text_column[index]
        text = " ".join(str(text).split())
        
        # get tokenized text
        tokenized_text = tokenizer.tokenize(text)
        
                
        # use the encode plus function
        dictionary_inputs = tokenizer.encode_plus(sentinent, text, 
                                                  max_length = max_seq_length,
                                                  #truncation_strategy  ='do_not_truncate', 
                                                  add_special_tokens = True,
                                                  pad_to_max_length = True, padding_side='Right', 
                                                  is_pretokenized = False, return_token_type_ids=True,
                                                  return_attention_mask=True,
                                                  return_offsets_mapping = True,
                                                  return_tensors = 'pt')
        
        return [dictionary_inputs, text_id]
  
    def __len__(self):
        return len(self.text_column)

In [56]:
# create a class to process the traininga and test data
batch_size = 4
test_data = Tweet_Loader_Test(test['text'],test['sentiment'], test['textID'])

# use the dataloaders class to load the data
dataloaders_dict_test = {'test': DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)}

dataset_sizes = {'test':len(test_data)}

In [57]:
def predictions(model = None):
    prediction_data_frame_list = []
    
    with torch.no_grad():
        model.eval()
        
        for i, (dict_param, tweet_id) in enumerate(dataloaders_dict_test['test']):
            # add a counter that will register how many examples we have fed to the
            # model
            # move the sequences, labels and masks to the GPU
                inputs = dict_param.get('input_ids').to(device).squeeze()
                token_type_ids = dict_param.get('token_type_ids').to(device).squeeze()
                attention_mask = dict_param.get('attention_mask').to(device).squeeze()

            # feed the sequences to the model, specifying the attention mask
                start_scores, end_scores = model(inputs, token_type_ids = token_type_ids, 
                                                   attention_mask = attention_mask)
            
                start_preds=torch.argmax(start_scores,dim=1).cpu().detach().numpy()
                end_preds=torch.argmax(end_scores,dim=1).cpu().detach().numpy()
                inputs = inputs.cpu().detach().numpy()
                
                for i in range(inputs.shape[0]):
                    a = tweet_id[i]
                    pred_string = reconstruct_string(start_preds[i], end_preds[i], inputs[i])
                    #pred_string = pred_string.replace(' ##', '')
                    print(f'the id is {a} and the string is {pred_string}')
                                        
                    temp_data = pd.DataFrame(zip([a],[pred_string]), columns = ['textID', 'selected_text'])
                    print(temp_data.head())
                    prediction_data_frame_list.append(temp_data)                
    
    prediction_df = pd.concat(prediction_data_frame_list)
    return prediction_df

In [36]:
model.load_state_dict(torch.load('../models_trained/bert_tweet_extraction_no_finetuned.pth'))


<All keys matched successfully>

In [59]:
prediction = predictions(model)

the id is f87dea47db and the string is last session of the day
       textID            selected_text
0  f87dea47db  last session of the day
the id is 96d74cb729 and the string is good tweeps
       textID selected_text
0  96d74cb729   good tweeps
the id is eee518ae67 and the string is shame!
       textID selected_text
0  eee518ae67        shame!
the id is 01082688c6 and the string is happy bday!
       textID selected_text
0  01082688c6   happy bday!
the id is 33987a8ee5 and the string is i like it!!
       textID selected_text
0  33987a8ee5   i like it!!
the id is 726e501993 and the string is that ` s great!
       textID    selected_text
0  726e501993  that ` s great!
the id is 261932614e and the string is hates
       textID selected_text
0  261932614e         hates
the id is afa11da83f and the string is blocked
       textID selected_text
0  afa11da83f       blocked
the id is e64208b4ef and the string is and within a short time of the last clue all of them
       textID          

the id is 614aed9113 and the string is tired..
       textID selected_text
0  614aed9113       tired..
the id is dfa577d3c1 and the string is thanks
       textID selected_text
0  dfa577d3c1        thanks
the id is 10dcbe4dfb and the string is you should totally come get me and bring me to kelslaws house with you.
       textID                                      selected_text
0  10dcbe4dfb  you should totally come get me and bring me to...
the id is a0bfb12773 and the string is you seem nice,
       textID   selected_text
0  a0bfb12773  you seem nice,
the id is 71f4a9b495 and the string is u so dirrrrrty... why oh why did i pick the hawks i figured i needed a longshot and they were a better bet than car
       textID                                      selected_text
0  71f4a9b495  u so dirrrrrty... why oh why did i pick the ha...
the id is d33ed264b3 and the string is almost got a kitty yesterday... but it didn ` t work out
       textID                                      selected

the id is 8f8334ffa2 and the string is and - hand made by dan & jenn - sock blockers... made from exotic woods... one of which is called'purple heart '
       textID                                      selected_text
0  8f8334ffa2  and - hand made by dan & jenn - sock blockers....
the id is 322546026e and the string is yay
       textID selected_text
0  322546026e           yay
the id is e6c154f9be and the string is sadly
       textID selected_text
0  e6c154f9be         sadly
the id is e96d849b35 and the string is tis awesome
       textID selected_text
0  e96d849b35   tis awesome
the id is 124e30d941 and the string is happy
       textID selected_text
0  124e30d941         happy
the id is fbb5742bbb and the string is _ adel they didn ` t reply yet.. wait till they confirm.. in case somebody might have translated it already..
       textID                                      selected_text
0  fbb5742bbb  _ adel they didn ` t reply yet.. wait till the...
the id is e90acf3c42 and the st

the id is 5ff525b74b and the string is wondering if i cld make things any worse than they already are... answer.. just did!!!
       textID                                      selected_text
0  5ff525b74b  wondering if i cld make things any worse than ...
the id is 4fd80e66e0 and the string is a dutch marketing agency is asking job applicants to apply in 140 characters or less
       textID                                      selected_text
0  4fd80e66e0  a dutch marketing agency is asking job applica...
the id is e77a8f8f2d and the string is frustratingly
       textID  selected_text
0  e77a8f8f2d  frustratingly
the id is 4ea7102112 and the string is i completely agree
       textID       selected_text
0  4ea7102112  i completely agree
the id is 6a14956779 and the string is if we don ` t pack, she can ` t leave, right?
       textID                                  selected_text
0  6a14956779  if we don ` t pack, she can ` t leave, right?
the id is 7c2fcb712e and the string is * * * *

the id is 9fae9e3145 and the string is good luck!
       textID selected_text
0  9fae9e3145    good luck!
the id is e515132ae4 and the string is thanks... i really do appreciate that. i know it will be the hardest thing i have ever done
       textID                                      selected_text
0  e515132ae4  thanks... i really do appreciate that. i know ...
the id is df3c7485ae and the string is i ` ll miss you guys!
       textID          selected_text
0  df3c7485ae  i ` ll miss you guys!
the id is bd8ba2ad7e and the string is i just love every little thing about you...
       textID                                selected_text
0  bd8ba2ad7e  i just love every little thing about you...
the id is 58be40735c and the string is stoooopid thing
       textID    selected_text
0  58be40735c  stoooopid thing
the id is b5a0a16fd7 and the string is hi simon, let me know when it ` s done
       textID                           selected_text
0  b5a0a16fd7  hi simon, let me know when it ` s

...

the id is a35314257a and the string is edgefest! or maybe since you ` re driving...
       textID                                 selected_text
0  a35314257a  edgefest! or maybe since you ` re driving...
the id is 2641ac37a6 and the string is sweetie!
       textID selected_text
0  2641ac37a6      sweetie!
the id is 7a1556637a and the string is that ` s alright. add your egg and maybe some sort of leftover meat and it ` s not so bad < 3
       textID                                      selected_text
0  7a1556637a  that ` s alright. add your egg and maybe some ...
the id is 08c5eb64a5 and the string is sorry
       textID selected_text
0  08c5eb64a5         sorry
the id is 6397544955 and the string is excellent
       textID selected_text
0  6397544955     excellent
the id is afc326749d and the string is sucks.
       textID selected_text
0  afc326749d        sucks.
the id is 79f88c9c67 and the string is oh no.... sorry
       textID    selected_text
0  79f88c9c67  oh no.... sorry
the 

the id is 3cdfcc7a2c and the string is it ` s not a reg gig yet, but hopefully it will be! haven ` t done massage in a while actually
       textID                                      selected_text
0  3cdfcc7a2c  it ` s not a reg gig yet, but hopefully it wil...
the id is 09554aef59 and the string is about 60 again todayy yayy!!!
       textID                  selected_text
0  09554aef59  about 60 again todayy yayy!!!
the id is c25f1544bd and the string is ashamed
       textID selected_text
0  c25f1544bd       ashamed
the id is 32cde6dbc5 and the string is miss you
       textID selected_text
0  32cde6dbc5      miss you
the id is 4e1101e520 and the string is i don ` t like it
       textID      selected_text
0  4e1101e520  i don ` t like it
the id is ed58e3db1d and the string is disappointed
       textID selected_text
0  ed58e3db1d  disappointed
the id is daba837f14 and the string is i didn ` t even get a chance to shop @ aldi!
       textID                                 selected_

the id is b8547d3835 and the string is yup on to mother ` s day. any plans for your birthday?
       textID                                      selected_text
0  b8547d3835  yup on to mother ` s day. any plans for your b...
the id is ed036f1d74 and the string is problem
       textID selected_text
0  ed036f1d74       problem
the id is a237d86171 and the string is _ * * * * r. i. p curls
       textID            selected_text
0  a237d86171  _ * * * * r. i. p curls
the id is 82dd0bcffe and the string is pretty
       textID selected_text
0  82dd0bcffe        pretty
the id is 89de64f532 and the string is can you get catacombs ffrom the cinemas? i tried to get it from my local one a little while ago and theyh didn ` t have it.
       textID                                      selected_text
0  89de64f532  can you get catacombs ffrom the cinemas? i tri...
the id is e0f51840d2 and the string is awesome
       textID selected_text
0  e0f51840d2       awesome
the id is 20f387fede and the strin

the id is 863e09ee74 and the string is have fun! i am sad to not be with you guys
       textID                               selected_text
0  863e09ee74  have fun! i am sad to not be with you guys
the id is 9501aabea3 and the string is died
       textID selected_text
0  9501aabea3          died
the id is 40ada63a36 and the string is have a huge block of snack chocolate here. tis calling my name already
       textID                                      selected_text
0  40ada63a36  have a huge block of snack chocolate here. tis...
the id is c956722677 and the string is my room is too hot
       textID       selected_text
0  c956722677  my room is too hot
the id is 1c6ce0a7e3 and the string is yes.
       textID selected_text
0  1c6ce0a7e3          yes.
the id is 5648799bc4 and the string is _ ellen and she ` s 9. 5 and a rescue.. bad hips. she deserves pampering! she ` s full of life though!!
       textID                                      selected_text
0  5648799bc4  _ ellen and s

the id is df1baec676 and the string is i love your videos!
       textID        selected_text
0  df1baec676  i love your videos!
the id is 469e15c5a8 and the string is omgssh ang cute
       textID    selected_text
0  469e15c5a8  omgssh ang cute


In [60]:
# load model
prediction.to_csv('submission.csv', index=False)

In [ ]:
#tokenizer.save_vocabulary('../models_trained/vocab.txt')

In [ ]:
"""
def targets_(text, selected_text):
    text = " ".join(str(text).split())
    selected_text = " ".join(str(selected_text).split())
    len_st= len(selected_text)
    idx0 = None
    idx1 = None
    for ind in (i for i, e in enumerate(text) if e == selected_text[0]):
        
        if text[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(text)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode_plus(text, return_offsets_mapping = True)
    
    tweet_offsets = tok_tweet.get('offset_mapping')[1:-1]
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    target_start = target_idx[0]
    targets_end = target_idx[-1]
    filtered_output = ''
    tweet_offsets
    for ix in range(target_start, targets_end + 1):
        #print(text[tweet_offsets[ix][0]:tweet_offsets[ix][1]])
        filtered_output += text[tweet_offsets[ix][0]:tweet_offsets[ix][1]]
        if (ix+1) < len(tweet_offsets) and tweet_offsets[ix][1] < tweet_offsets[ix+1][0]:
            filtered_output += " "
    print(filtered_output)
    return target_start, targets_end, tok_tweet
"""